<a href="https://colab.research.google.com/github/Tristan-Brown1096/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv')).set_index('id')
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv', index_col='id')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 40), (14358, 39))

In [3]:
train.date_recorded = pd.to_datetime(train.date_recorded)
test.date_recorded = pd.to_datetime(test.date_recorded)

In [4]:
import numpy as np
train.construction_year = train.construction_year.replace(0, np.nan)
test.construction_year = test.construction_year.replace(0, np.nan)

In [5]:
train['years_between_inspection'] = train['date_recorded'].dt.year - train['construction_year']
test['years_between_inspection'] = test['date_recorded'].dt.year - test['construction_year']

In [6]:
def wrangle(X):
  #copy X
  X = X.copy()

  #replace weird latitude values
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  #replace lat/long 0 values with NaN
  cols_with_zeros = ['longitude', 'latitude']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)

  #drop constant columns
  X = X.drop([col for col in X if X[col].nunique()==1], axis=1)

  #drop high cardinality columns
  hc = [col for col in X.describe(include='object').columns if X[col].nunique() > 100]
  X = X.drop(hc, axis=1)

  #drop repeated column
  X = X.drop(['quantity_group', 'date_recorded', 'construction_year'], axis=1)

  #return new version 
  return X

In [7]:
train = wrangle(train)
test = wrangle(test)

#Split data

In [8]:
y = train['status_group']

X_2 = train.drop(['status_group', 'scheme_management', 'permit', 'source', 'basin', 'management', 'permit', 'public_meeting'], axis=1)
X = train.drop('status_group', axis=1)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 42)

#Build model_21

In [10]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [11]:
model_21 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    SelectKBest(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_features='sqrt', max_depth=20)
    
)

model_21.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=20, max_features='sqrt',
                                   

In [12]:
print('Training accuracy', model_21.score(X_train, y_train))
print('Validation accuracy', model_21.score(X_val, y_val))

Training accuracy 0.878030303030303
Validation accuracy 0.7644781144781144


#Tune model

In [14]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [17]:
params = {'selectkbest__k': range(1, 10, 2),
          'randomforestclassifier__n_estimators': range(250, 500, 50),
          'randomforestclassifier__max_depth': range(1, 10, 2),
}

gs = GridSearchCV(model_21, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [18]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 625 out of 625 | elapsed: 38.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=['basin', 'region',
                                                             'public_meeting',
                                                             'scheme_management',
                                                             'permit',
                                                             'extraction_type',
                                                             'extraction_type_group',
                                                             'extraction_type_class',
                                                             'management',
                                                             'management_group',
                                                             'payment',
                                                             'payment

In [19]:
gs.best_params_

{'randomforestclassifier__max_depth': 9,
 'randomforestclassifier__n_estimators': 400,
 'selectkbest__k': 9}

#Build new model

In [20]:
model_22 = gs.best_estimator_

In [21]:
print('Training accuracy', model_22.score(X_train, y_train))
print('Validation accuracy', model_22.score(X_val, y_val))

Training accuracy 0.7432449494949495
Validation accuracy 0.7412457912457913


#Build model_23

In [23]:
model_23 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=400, max_features='sqrt', max_depth=9)
    
)

model_23.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=9, max_features='sqrt',
                                    

In [25]:
print('Training accuracy', model_23.score(X_train, y_train))
print('Validation accuracy', model_23.score(X_val, y_val))

Training accuracy 0.7610479797979798
Validation accuracy 0.7536195286195286


#Tune model again

In [33]:
params = {'randomforestclassifier__n_estimators': range(300, 500, 50),
          'randomforestclassifier__max_depth': range(1, 10, 2),
          'randomforestclassifier__max_features': ['auto', 'log2', None]
}

gs = GridSearchCV(model_23, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [35]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 34.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 78.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=['basin', 'region',
                                                             'public_meeting',
                                                             'scheme_management',
                                                             'permit',
                                                             'extraction_type',
                                                             'extraction_type_group',
                                                             'extraction_type_class',
                                                             'management',
                                                             'management_group',
                                                             'payment',
                                                             'payment

In [36]:
gs.best_params_

{'randomforestclassifier__max_depth': 9,
 'randomforestclassifier__max_features': 'auto',
 'randomforestclassifier__n_estimators': 400}

#Build model_24

In [38]:
model_24 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=400, max_features='auto', max_depth=9)
    
)

model_24.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=9, max_features='auto',
                                    

In [39]:
print('Training accuracy', model_24.score(X_train, y_train))
print('Validation accuracy', model_24.score(X_val, y_val))

Training accuracy 0.7610479797979798
Validation accuracy 0.7536195286195286


#Tune model again

In [43]:
params = {'randomforestclassifier__max_depth': range(9, 21),
}

gs = GridSearchCV(model_24, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [44]:
gs.fit(X_train, y_train)
gs.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 13.2min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 18.1min finished


{'randomforestclassifier__max_depth': 19}

#Build model_25

In [45]:
model_25 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=400, max_features='auto', max_depth=19)
    
)

model_25.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=19, max_features='auto',
                                   

In [46]:
print('Training accuracy', model_25.score(X_train, y_train))
print('Validation accuracy', model_25.score(X_val, y_val))

Training accuracy 0.9335437710437711
Validation accuracy 0.8085858585858586


#Test model_25

In [49]:
from google.colab import files
X_test = test
y_pred_9 = model_25.predict(X_test)
y_pred_9 = pd.DataFrame(y_pred_9, columns=['status_group'], index=X_test.index)
y_pred_9.to_csv('tristan_brown_submit_9.csv')
files.download('tristan_brown_submit_9.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Build model_26

In [50]:
model_26 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_features='auto', max_depth=19)
    
)

model_26.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=19, max_features='auto',
                                   

In [51]:
print('Training accuracy', model_26.score(X_train, y_train))
print('Validation accuracy', model_26.score(X_val, y_val))

Training accuracy 0.9332491582491582
Validation accuracy 0.8082491582491582


#Test model_26

In [52]:
y_pred_10 = model_26.predict(X_test)
y_pred_10 = pd.DataFrame(y_pred_10, columns=['status_group'], index=X_test.index)
y_pred_10.to_csv('tristan_brown_submit_10.csv')
files.download('tristan_brown_submit_10.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>